In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
univ = pd.read_csv('univtopro_preprocessed.csv')
pro = pd.read_csv('KBL.csv')

In [3]:
univ_avg = univ.groupby(univ['선수명']).mean()
pro_avg = pro.groupby(pro['선수명']).mean()

In [4]:
data = pd.merge(univ_avg, pro_avg, how='inner', left_index=True, right_index=True)
data

,Unnamed: 0_x,경기수,전체 출전시간,득점 전체,득점 평균,2점슛 성공,2점슛 시도,2점슛 성공률,3점슛 성공,3점슛 시도,...,PPA,PP%,REB_OFF,REB_DEF,REB_TOT,AST,TO,STL,BS,DF
선수명,,,,,,,,,,,,,,,,,,,,,
강권희,1.0,2.333333,2442.666667,12.666667,6.333333,2.666667,6.333333,666.666667,2.333333,7.666667,...,0.000000,0.000000,0.350000,2.100000,2.425000,0.775000,1.000000,0.700000,0.225000,2.425000
강병진,4.0,3.333333,2964.666667,10.333333,2.333333,3.000000,7.333333,429.333333,1.333333,7.333333,...,0.000000,0.000000,0.650000,1.150000,1.800000,0.600000,1.250000,0.500000,0.150000,2.250000
강병현,11.5,8.333333,8493.000000,62.583333,8.416667,18.333333,33.000000,548.166667,5.333333,17.500000,...,1.467857,38.050000,0.475000,1.478571,1.964286,1.339286,0.900000,0.364286,0.067857,1.592857
강상재,21.5,9.625000,12136.500000,104.500000,9.875000,41.250000,73.625000,531.000000,4.250000,12.875000,...,3.658333,68.558333,1.366667,3.391667,4.766667,1.233333,0.950000,0.541667,0.325000,2.100000
강우형,27.0,5.333333,6621.333333,23.666667,3.000000,8.333333,10.666667,704.666667,1.666667,4.000000,...,0.300000,21.450000,0.350000,0.200000,0.550000,0.250000,0.150000,0.150000,0.000000,0.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
홍석민,2560.0,8.222222,5923.000000,30.000000,3.222222,9.666667,20.111111,453.888889,3.333333,10.222222,...,0.000000,0.000000,0.000000,0.400000,0.400000,0.000000,0.000000,0.000000,0.000000,0.500000
홍성헌,2566.0,4.000000,7204.333333,49.666667,12.000000,19.333333,46.000000,463.000000,0.000000,1.000000,...,2.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
홍수화,2569.5,4.250000,6719.500000,49.000000,11.000000,12.250000,24.000000,488.000000,6.000000,19.500000,...,0.300000,30.766667,0.466667,0.833333,1.300000,0.533333,0.233333,0.200000,0.033333,1.800000


In [5]:
#가산점 항목
KBL_eff_pos = (data['PTS'].to_numpy() + data['STL'].to_numpy() + data['BS'].to_numpy() + data['REB_DEF'].to_numpy()) * 1.0\
            + (data['REB_OFF'].to_numpy() + data['AST'].to_numpy()) * 1.5

#감점 항목
KBL_eff_neg = (data['TO'].to_numpy()) * 1.5\
            + (data['FGA'].to_numpy() - data['FG'].to_numpy()) * 1.0\
            + (data['PP'].to_numpy() - data['PP'].to_numpy()) * 1.0\
            + (data['FTA'].to_numpy() - data['FT'].to_numpy()) * 0.8\

KBL_eff = KBL_eff_pos - KBL_eff_neg

In [6]:
#-1과 1사이 수치로 정규화
#KBL_eff_max, KBL_eff_min = np.max(KBL_eff), np.min(KBL_eff)
#KBL_eff = KBL_eff / KBL_eff_max if abs(KBL_eff_max) > abs(KBL_eff_min) else KBL_eff / -KBL_eff_min

In [7]:
data['KBL_EFF'] = KBL_eff
data

,Unnamed: 0_x,경기수,전체 출전시간,득점 전체,득점 평균,2점슛 성공,2점슛 시도,2점슛 성공률,3점슛 성공,3점슛 시도,...,PP%,REB_OFF,REB_DEF,REB_TOT,AST,TO,STL,BS,DF,KBL_EFF
선수명,,,,,,,,,,,,,,,,,,,,,
강권희,1.0,2.333333,2442.666667,12.666667,6.333333,2.666667,6.333333,666.666667,2.333333,7.666667,...,0.000000,0.350000,2.100000,2.425000,0.775000,1.000000,0.700000,0.225000,2.425000,11.087500
강병진,4.0,3.333333,2964.666667,10.333333,2.333333,3.000000,7.333333,429.333333,1.333333,7.333333,...,0.000000,0.650000,1.150000,1.800000,0.600000,1.250000,0.500000,0.150000,2.250000,6.360000
강병현,11.5,8.333333,8493.000000,62.583333,8.416667,18.333333,33.000000,548.166667,5.333333,17.500000,...,38.050000,0.475000,1.478571,1.964286,1.339286,0.900000,0.364286,0.067857,1.592857,8.736429
강상재,21.5,9.625000,12136.500000,104.500000,9.875000,41.250000,73.625000,531.000000,4.250000,12.875000,...,68.558333,1.366667,3.391667,4.766667,1.233333,0.950000,0.541667,0.325000,2.100000,14.715000
강우형,27.0,5.333333,6621.333333,23.666667,3.000000,8.333333,10.666667,704.666667,1.666667,4.000000,...,21.450000,0.350000,0.200000,0.550000,0.250000,0.150000,0.150000,0.000000,0.650000,1.145000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
홍석민,2560.0,8.222222,5923.000000,30.000000,3.222222,9.666667,20.111111,453.888889,3.333333,10.222222,...,0.000000,0.000000,0.400000,0.400000,0.000000,0.000000,0.000000,0.000000,0.500000,-0.250000
홍성헌,2566.0,4.000000,7204.333333,49.666667,12.000000,19.333333,46.000000,463.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.500000
홍수화,2569.5,4.250000,6719.500000,49.000000,11.000000,12.250000,24.000000,488.000000,6.000000,19.500000,...,30.766667,0.466667,0.833333,1.300000,0.533333,0.233333,0.200000,0.033333,1.800000,4.963333


In [8]:
X = data[['경기수', '전체 출전시간', '득점 전체', '득점 평균', '2점슛 성공', '2점슛 시도',
       '2점슛 성공률', '3점슛 성공', '3점슛 시도', '3점슛 성공률', '자유투 성공', '자유투 시도', '자유투 성공률',
       '공격 리바운드', '수비 리바운드', '평균 리바운드', '전체 어시스트', '평균 어시스트', '스틸', '블록',
       '덩크슛 성공', '덩크슛 시도', '파울(자유튜유)', '파울(자유투무)', '테크니컬 파울', '굿디펜스',
       '속공', 'FBF', '턴오버', '팀 리바운드', '팀 턴오버']]

y = data['KBL_EFF']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
y_pred = linear_regressor.predict(X_test)
r2_score(y_pred, y_test)

-1.556271327001701